In [1]:
conda activate reco_base


Note: you may need to restart the kernel to use updated packages.


In [2]:
cd D:\Algorithmic Marketing\Assignment 3\recommenders

D:\Algorithmic Marketing\Assignment 3\recommenders


In [3]:
import sys
sys.path.append("../../")
import os
import pickle
import papermill as pm
import pandas as pd
import numpy as np
import tensorflow as tf
from reco_utils.common.timer import Timer
from reco_utils.recommender.deeprec.models.graphrec.lightgcn import LightGCN
from reco_utils.recommender.deeprec.DataModel.ImplicitCF import ImplicitCF
from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_stratified_split
from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from reco_utils.common.constants import SEED as DEFAULT_SEED
from reco_utils.recommender.deeprec.deeprec_utils import prepare_hparams


print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
Pandas version: 1.0.5
Tensorflow version: 1.15.2


In [4]:
# top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
ALIBABA_DATA_SIZE = '100k'

# Model parameters
EPOCHS = 50
BATCH_SIZE = 1024

SEED = DEFAULT_SEED  # Set None for non-deterministic results

yaml_file = "D:/Algorithmic Marketing/Assignment 3/recommenders/reco_utils/recommender/deeprec/config/lightgcn.yaml"
user_file = "../../tests/resources/deeprec/lightgcn/user_embeddings.csv"
item_file = "../../tests/resources/deeprec/lightgcn/item_embeddings.csv"

In [5]:
df = pd.read_csv("D:/Algorithmic Marketing/Final Project/Dataset/Recommendation_data.csv")

In [6]:
df.head()

,userID,itemID,rating
0,17850,85123A,8
1,17850,71053,7
2,17850,84406B,8
3,17850,84029G,5
4,17850,84029E,9


In [7]:
train, test = python_stratified_split(df, ratio=0.75)

In [8]:
data = ImplicitCF(train=train, test=test, seed=SEED)

In [9]:
hparams = prepare_hparams(yaml_file,
                          n_layers=3,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,
                         )

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [10]:
model = LightGCN(hparams, data, seed=SEED)


Already create adjacency matrix.
Already normalize adjacency matrix.

Using xavier initialization.




D:\Algorithmic Marketing\Assignment 3\recommenders\reco_utils\recommender\deeprec\DataModel\ImplicitCF.py:179: RuntimeWarning: divide by zero encountered in power
  d_inv = np.power(rowsum, -0.5).flatten()


In [ ]:
pickle.dump( model, open( "LGCN_model.p", "wb" ) )

In [11]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))

Epoch 1 (train)2.7s: train loss = 0.58990 = (mf)0.58975 + (embed)0.00015
Epoch 2 (train)2.4s: train loss = 0.38810 = (mf)0.38742 + (embed)0.00067
Epoch 3 (train)2.3s: train loss = 0.33910 = (mf)0.33811 + (embed)0.00099
Epoch 4 (train)2.2s: train loss = 0.29073 = (mf)0.28941 + (embed)0.00133
Epoch 5 (train)2.3s + (eval)0.3s: train loss = 0.25533 = (mf)0.25363 + (embed)0.00170, recall = 0.03931, ndcg = 0.05804, precision = 0.04450, map = 0.01551
Epoch 6 (train)2.3s: train loss = 0.22173 = (mf)0.21965 + (embed)0.00208
Epoch 7 (train)2.3s: train loss = 0.19596 = (mf)0.19351 + (embed)0.00245
Epoch 8 (train)2.3s: train loss = 0.17456 = (mf)0.17174 + (embed)0.00282
Epoch 9 (train)2.3s: train loss = 0.15914 = (mf)0.15597 + (embed)0.00317
Epoch 10 (train)2.3s + (eval)0.2s: train loss = 0.14731 = (mf)0.14381 + (embed)0.00350, recall = 0.05986, ndcg = 0.08540, precision = 0.06323, map = 0.02454
Epoch 11 (train)2.3s: train loss = 0.13399 = (mf)0.13016 + (embed)0.00383
Epoch 12 (train)2.3s: train l

In [12]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

topk_scores.head()

,userID,itemID,prediction
0,12349,23245,8.174675
1,12349,22839,7.465651
2,12349,22423,7.303855
3,12349,23284,6.950679
4,12349,22507,6.786175


In [13]:
eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.041018
NDCG:	0.124642
Precision@K:	0.088660
Recall@K:	0.090790


In [ ]:
pm.record("map", eval_map)
pm.record("ndcg", eval_ndcg)
pm.record("precision", eval_precision)
pm.record("recall", eval_recall)

In [14]:
model.infer_embedding(user_file, item_file)

In [14]:
cd C:\Users\verle\Assignment_3_Recommendation

C:\Users\verle\Assignment_3_Recommendation


In [15]:
import joblib

In [17]:
filename = 'LightGCN.sav'
joblib.dump(topk_scores, filename)

['LightGCN.sav']